In [61]:
from __future__ import division
from subprocess import call
import pandas as pd

### 1. 确定url

In [38]:
filename = '000300closeweight.xls'
# filename = '000905closeweight.xls'
urlPrefix = 'ftp://115.29.204.48/webdata/'
url = urlPrefix + filename
url

'ftp://115.29.204.48/webdata/000300closeweight.xls'

### 2. 下载文件

In [37]:
call(['rm', filename])
result = call(['wget', url])
print result 

0


### 3. 加载到内存

In [63]:
df = pd.io.excel.read_excel('000300closeweight.xls')
df = df[['Date', 'Constituent Code', 'Weight(%)']]
df.columns = ['date', 'code', 'weight']
df['code'] = df.code.apply(lambda x: str(x).zfill(6))
df['weight']  = df['weight'] / 100
df.head()

,date,code,weight
0,2016-03-01,000001,0.0072
1,2016-03-01,000002,0.0245
2,2016-03-01,000009,0.0019
3,2016-03-01,000027,0.0010
4,2016-03-01,000039,0.0013


### 4. 保存在数据库